In [1]:
import os
import time
import random

import numpy as np
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

import pickle

In [2]:
images_dir = './PetImages'
categories = ['Dog', 'Cat']
img_size = 70
flag_to_channel_count = {
    # grayscale pixel: 0-255
    # rgb pixel: [0-255, 0-255, 0-255]
    cv2.IMREAD_GRAYSCALE: 1,
    cv2.IMREAD_COLOR: 3
}

# Preprocessing

In [3]:
def create_training_data(flag):
    training_data = []
    channel_count = flag_to_channel_count[flag]
    for category in categories:
        path = os.path.join(images_dir, category)
        class_num = categories.index(category) # dog == 0, cat == 1
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), flag)
                resized_img_arr = cv2.resize(img_arr, (img_size, img_size))
                training_data.append([resized_img_arr, class_num])
            except Exception as e:
                pass
            
    random.shuffle(training_data)
        
    X = []
    y = []

    for features, label in training_data:
        X.append(features)
        y.append(label)
    
    # for some reason regular lists dont work so we convert to np arrays
    X = np.array(X).reshape(-1, img_size, img_size, channel_count)
    y = np.array(y)
    
    print(f"successfully processed {len(training_data)} photos")
    
    return (X, y)

X_gray, y_gray = create_training_data(cv2.IMREAD_GRAYSCALE)
X_rgb, y_rgb = create_training_data(cv2.IMREAD_COLOR)

successfully processed 24946 photos
successfully processed 24946 photos


In [4]:
# pickle dump to save for later use
pickle_out = open("X_gray.pickle", "wb")
pickle.dump(X_gray, pickle_out)
pickle_out.close()

pickle_out = open("y_gray.pickle", "wb")
pickle.dump(y_gray, pickle_out)
pickle_out.close()

pickle_out = open("X_rgb.pickle", "wb")
pickle.dump(X_rgb, pickle_out)
pickle_out.close()

pickle_out = open("y_rgb.pickle", "wb")
pickle.dump(y_rgb, pickle_out)
pickle_out.close()

In [ ]:
if (len(tf.config.list_physical_devices('GPU')) == 0):
    print('gpu not detected, training will be slow')

# Model optimization
## Test different dense layers, layer sizes, conv layers (grayscale)

In [ ]:
X_gray = pickle.load(open("X_gray.pickle", "rb"))
y_gray = pickle.load(open("y_gray.pickle", "rb"))
X_gray = X_gray / 255.0

dense_layer_options = [0, 1, 2]
layer_size_options = [32, 64, 128]
conv_layer_options = [1, 2, 3]


for dense_layer in dense_layer_options:
    for layer_size in layer_size_options:
        for conv_layer in conv_layer_options:
            model_name = f"{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense-{int(time.time())}"
            tensorboard = TensorBoard(log_dir="logs/{}".format(model_name))

            optimized_model = Sequential()
            
            optimized_model.add(Conv2D(layer_size, (3,3), input_shape = X_gray.shape[1:]))
            optimized_model.add(Activation("relu"))
            optimized_model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                optimized_model.add(Conv2D(layer_size, (3,3)))
                optimized_model.add(Activation("relu"))
                optimized_model.add(MaxPooling2D(pool_size=(2,2)))

            optimized_model.add(Flatten())
            
            for l in range(dense_layer):
                optimized_model.add(Dense(layer_size))
                optimized_model.add(Activation("relu"))

            optimized_model.add(Dense(1))
            optimized_model.add(Activation("sigmoid"))

            optimized_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
            optimized_model.fit(X_gray, y_gray, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

## Result: 3 conv 0 dense 128 layer size had least eval loss

## Try higher layer size options

In [ ]:
X_gray = pickle.load(open("X_gray.pickle", "rb"))
y_gray = pickle.load(open("y_gray.pickle", "rb"))

X_gray = X_gray / 255.0
layer_size_options = [256, 512, 1024]


for layer_size in layer_size_options:
    model_name = f"3-conv-{layer_size}-nodes-0-dense-{int(time.time())}"
    tensorboard = TensorBoard(log_dir="logs/{}".format(model_name))

    optimized_model = Sequential()

    optimized_model.add(Conv2D(layer_size, (3,3), input_shape = X_gray.shape[1:]))
    optimized_model.add(Activation("relu"))
    optimized_model.add(MaxPooling2D(pool_size=(2,2)))

    optimized_model.add(Conv2D(layer_size, (3,3)))
    optimized_model.add(Activation("relu"))
    optimized_model.add(MaxPooling2D(pool_size=(2,2)))

    optimized_model.add(Conv2D(layer_size, (3,3)))
    optimized_model.add(Activation("relu"))
    optimized_model.add(MaxPooling2D(pool_size=(2,2)))

    optimized_model.add(Flatten())

    optimized_model.add(Dense(1))
    optimized_model.add(Activation("sigmoid"))

    optimized_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    optimized_model.fit(X_gray, y_gray, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

## Final results: 3 conv layers 64 layer size 0 dense

# Final model structure

In [5]:
def get_final_model(X):
    
    model = Sequential()

    model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())

    model.add(Dense(1))
    model.add(Activation("sigmoid"))

    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model

# Trying to see if rgb works better than grayscale

In [ ]:
X_gray = pickle.load(open("X_gray.pickle", "rb"))
y_gray = pickle.load(open("y_gray.pickle", "rb"))

X_gray = X_gray / 255.0

tensorboard = TensorBoard(log_dir="logs/{3-conv-64-nodes-0-dense-gray}")
final_model_gray = get_final_model(X_gray)
final_model_gray.fit(X_gray, y_gray, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

model.save('./classifier-api/64x3-CNN.model')

In [ ]:
X_rgb = pickle.load(open("X_rgb.pickle", "rb"))
y_rgb = pickle.load(open("y_rgb.pickle", "rb"))

X_rgb = X_rgb / 255.0

tensorboard = TensorBoard(log_dir="logs/{3-conv-64-nodes-0-dense-rgb}")
final_model_rgb = get_final_model(X_rgb)
final_model_rgb.fit(X_rgb, y_rgb, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

model.save('./classifier-api/64x3-CNN-rgb.model')

# Experimenting with the effects of normalized image size

In [ ]:
img_size = 120
X_gray_high_res, y_gray_high_res = create_training_data(cv2.IMREAD_GRAYSCALE)
X_gray_high_res = X_gray_high_res / 255.0

tensorboard = TensorBoard(log_dir="logs/{3-conv-64-nodes-0-dense-grey-high-res}")
earlystopping = EarlyStopping(monitor ="val_loss", 
                              mode ="min",
                              patience = 5, 
                              restore_best_weights = True)
final_model_gray_high_res = get_final_model(X_gray_high_res)
final_model_gray_high_res.fit(
    X_gray_high_res,
    y_gray_high_res,
    batch_size=32,
    epochs=6,
    validation_split=0.1,
    callbacks=[tensorboard, earlystopping])

model.save('./classifier-api/64x3-CNN-grey-high-res.model')


In [ ]:
# literally not enough ram to run this

# img_size = 120
# X_rgb_high_res, y_rgb_high_res = create_training_data(cv2.IMREAD_COLOR)
# X_rgb_high_res = X_rgb_high_res / 255.0

# tensorboard = TensorBoard(log_dir="logs/{3-conv-64-nodes-0-dense-rgb-high-res}")
# earlystopping = EarlyStopping(monitor ="val_loss", 
#                               mode ="min",
#                               patience = 5, 
#                               restore_best_weights = True)
# final_model_rgb_high_res = get_final_model(X_rgb_high_res)
# final_model_rgb_high_res.fit(
#     X_rgb_high_res,
#     X_rgb_high_res,
#     batch_size=32,
#     epochs=15,
#     validation_split=0.1,
#     callbacks=[tensorboard, earlystopping])

# model.save('./classifier-api/64x3-CNN-rgb-high-res.model')